In [2]:
# Insurance Agent Performance EDA

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

# Read the training data
train_df = pd.read_csv('train_stroming_round.csv')

# Display basic information about the dataset
print("Dataset shape:", train_df.shape)
print("\nFirst few rows of the dataset:")
display(train_df.head())

# Check for missing values
print("\nMissing values in each column:")
print(train_df.isnull().sum())

# Check data types
print("\nData types:")
print(train_df.dtypes)

# Create the target column as specified in the instructions
train_df['target'] = (train_df['new_policy_count'] > 0).astype(int)
print("\nTarget distribution (0 = NILL, 1 = Active):")
print(train_df['target'].value_counts())
print(f"Percentage of NILL agents: {(1 - train_df['target'].mean()) * 100:.2f}%")

# Convert date columns to datetime
date_columns = ['agent_join_month', 'first_policy_sold_month', 'year_month']
for col in date_columns:
    train_df[col] = pd.to_datetime(train_df[col])

# Extract additional temporal features
train_df['agent_tenure_months'] = ((train_df['year_month'].dt.year - train_df['agent_join_month'].dt.year) * 12 + 
                                  (train_df['year_month'].dt.month - train_df['agent_join_month'].dt.month))

train_df['months_since_first_sale'] = ((train_df['year_month'].dt.year - train_df['first_policy_sold_month'].dt.year) * 12 + 
                                      (train_df['year_month'].dt.month - train_df['first_policy_sold_month'].dt.month))

# Replace infinite values with NaN and then fill with 0
train_df.replace([np.inf, -np.inf], np.nan, inplace=True)
train_df['months_since_first_sale'].fillna(0, inplace=True)

# Add year and month columns for easier analysis
train_df['year'] = train_df['year_month'].dt.year
train_df['month'] = train_df['year_month'].dt.month

# Create conversion ratios
train_df['proposal_to_quotation_ratio'] = train_df['unique_quotations'] / train_df['unique_proposal'].replace(0, 1)
train_df['quotation_to_policy_ratio'] = train_df['new_policy_count'] / train_df['unique_quotations'].replace(0, 1)
train_df['proposal_to_policy_ratio'] = train_df['new_policy_count'] / train_df['unique_proposal'].replace(0, 1)
train_df['cash_payment_ratio'] = train_df['number_of_cash_payment_policies'] / train_df['number_of_policy_holders'].replace(0, 1)
train_df['avg_premium_per_policy'] = train_df['ANBP_value'] / train_df['new_policy_count'].replace(0, 1)

# ----------------------------------------------------------------------------------
# 1. KEY METRICS AND DISTRIBUTIONS (SUMMARY STATISTICS)
# ----------------------------------------------------------------------------------

print("\n\n========== 1. KEY METRICS AND DISTRIBUTIONS ==========")

# Get summary statistics for numerical columns
numerical_cols = train_df.select_dtypes(include=['int64', 'float64']).columns
print("\nSummary statistics for numerical features:")
display(train_df[numerical_cols].describe())

# Plot distributions of key performance metrics
plt.figure(figsize=(20, 15))

# Create a list of key performance metrics
key_metrics = ['new_policy_count', 'ANBP_value', 'net_income', 'number_of_policy_holders',
               'unique_proposal', 'unique_quotations', 'unique_customers']

for i, metric in enumerate(key_metrics):
    plt.subplot(3, 3, i+1)
    
    # Plot histogram with KDE
    sns.histplot(train_df[metric], kde=True)
    
    # Add median line
    median_val = train_df[metric].median()
    plt.axvline(median_val, color='red', linestyle='--', 
                label=f'Median: {median_val:.2f}')
    
    # Add mean line
    mean_val = train_df[metric].mean()
    plt.axvline(mean_val, color='green', linestyle='-', 
                label=f'Mean: {mean_val:.2f}')
    
    plt.title(f'Distribution of {metric}')
    plt.legend()
    plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('key_metrics_distributions.png')
plt.show()

# Plot distributions by target
plt.figure(figsize=(20, 15))

for i, metric in enumerate(key_metrics):
    plt.subplot(3, 3, i+1)
    
    sns.kdeplot(data=train_df, x=metric, hue='target', common_norm=False, fill=True)
    
    plt.title(f'Distribution of {metric} by Target')
    plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('key_metrics_by_target.png')
plt.show()

# Correlation matrix
plt.figure(figsize=(14, 12))
correlation_matrix = train_df[numerical_cols].corr()
mask = np.triu(correlation_matrix)
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            mask=mask, linewidths=0.5, vmin=-1, vmax=1)
plt.title('Correlation Matrix of Numerical Features')
plt.tight_layout()
plt.savefig('correlation_matrix.png')
plt.show()

# Distribution of agent ages
plt.figure(figsize=(12, 6))
sns.histplot(data=train_df, x='agent_age', hue='target', kde=True, element='step')
plt.title('Distribution of Agent Ages by Target')
plt.grid(True, alpha=0.3)
plt.savefig('agent_age_distribution.png')
plt.show()

# Distribution of agent tenure
plt.figure(figsize=(12, 6))
sns.histplot(data=train_df, x='agent_tenure_months', hue='target', kde=True, element='step')
plt.title('Distribution of Agent Tenure by Target')
plt.grid(True, alpha=0.3)
plt.savefig('agent_tenure_distribution.png')
plt.show()

# Distribution of conversion ratios
plt.figure(figsize=(16, 10))
conversion_ratios = ['proposal_to_quotation_ratio', 'quotation_to_policy_ratio', 
                    'proposal_to_policy_ratio', 'cash_payment_ratio']

for i, ratio in enumerate(conversion_ratios):
    plt.subplot(2, 2, i+1)
    sns.histplot(data=train_df, x=ratio, hue='target', kde=True, element='step')
    plt.title(f'Distribution of {ratio} by Target')
    plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('conversion_ratios.png')
plt.show()

# ----------------------------------------------------------------------------------
# 2. SALES PATTERNS BY MONTH AND TIME SERIES ANALYSIS
# ----------------------------------------------------------------------------------

print("\n\n========== 2. SALES PATTERNS BY MONTH ==========")

# Aggregate data by month
monthly_stats = train_df.groupby('year_month').agg({
    'new_policy_count': 'mean',
    'ANBP_value': 'mean',
    'net_income': 'mean',
    'target': 'mean'
}).reset_index()

monthly_stats['nill_rate'] = 1 - monthly_stats['target']

# Plot time series of key metrics
plt.figure(figsize=(16, 10))

# New policy count over time
plt.subplot(2, 2, 1)
plt.plot(monthly_stats['year_month'], monthly_stats['new_policy_count'], marker='o')
plt.title('Average New Policy Count Over Time')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

# ANBP value over time
plt.subplot(2, 2, 2)
plt.plot(monthly_stats['year_month'], monthly_stats['ANBP_value'], marker='o')
plt.title('Average ANBP Value Over Time')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

# Net income over time
plt.subplot(2, 2, 3)
plt.plot(monthly_stats['year_month'], monthly_stats['net_income'], marker='o')
plt.title('Average Net Income Over Time')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

# NILL rate over time
plt.subplot(2, 2, 4)
plt.plot(monthly_stats['year_month'], monthly_stats['nill_rate'], marker='o')
plt.title('NILL Rate Over Time')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('time_series_analysis.png')
plt.show()

# Seasonal decomposition
# First, let's check if we have enough data for seasonal decomposition
print("\nUnique year-month combinations:", train_df['year_month'].nunique())
unique_months = sorted(train_df['year_month'].unique())
print("Date range:", min(unique_months), "to", max(unique_months))

# Let's analyze seasonal patterns by month
monthly_patterns = train_df.groupby('month').agg({
    'new_policy_count': 'mean',
    'ANBP_value': 'mean',
    'net_income': 'mean',
    'target': 'mean'
}).reset_index()

monthly_patterns['nill_rate'] = 1 - monthly_patterns['target']

# Plot monthly patterns
plt.figure(figsize=(16, 10))

metrics = ['new_policy_count', 'ANBP_value', 'net_income', 'nill_rate']
titles = ['Average New Policy Count', 'Average ANBP Value', 'Average Net Income', 'NILL Rate']

for i, (metric, title) in enumerate(zip(metrics, titles)):
    plt.subplot(2, 2, i+1)
    plt.bar(monthly_patterns['month'], monthly_patterns[metric])
    plt.title(f'{title} by Month')
    plt.xlabel('Month')
    plt.xticks(range(1, 13))
    plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('monthly_patterns.png')
plt.show()

# Check for anomalies in the time series
fig = make_subplots(rows=2, cols=2, subplot_titles=('Avg New Policy Count', 'Avg ANBP Value', 
                                                  'Avg Net Income', 'NILL Rate'))

metrics = ['new_policy_count', 'ANBP_value', 'net_income', 'nill_rate']
row_col_pairs = [(1, 1), (1, 2), (2, 1), (2, 2)]

for (metric, (row, col)) in zip(metrics, row_col_pairs):
    # Calculate mean and standard deviation
    mean_val = monthly_stats[metric].mean()
    std_val = monthly_stats[metric].std()
    
    # Define anomaly thresholds (2 standard deviations from mean)
    upper_bound = mean_val + 2 * std_val
    lower_bound = mean_val - 2 * std_val
    
    # Identify anomalies
    anomalies = monthly_stats[
        (monthly_stats[metric] > upper_bound) | 
        (monthly_stats[metric] < lower_bound)
    ]
    
    # Add time series trace
    fig.add_trace(
        go.Scatter(
            x=monthly_stats['year_month'], 
            y=monthly_stats[metric],
            mode='lines+markers',
            name=metric
        ),
        row=row, col=col
    )
    
    # Add anomaly points
    if not anomalies.empty:
        fig.add_trace(
            go.Scatter(
                x=anomalies['year_month'],
                y=anomalies[metric],
                mode='markers',
                marker=dict(size=10, color='red'),
                name=f'{metric} anomalies'
            ),
            row=row, col=col
        )
    
    # Add threshold lines
    fig.add_shape(
        type="line",
        x0=min(monthly_stats['year_month']),
        y0=upper_bound,
        x1=max(monthly_stats['year_month']),
        y1=upper_bound,
        line=dict(color="red", width=1, dash="dash"),
        row=row, col=col
    )
    
    fig.add_shape(
        type="line",
        x0=min(monthly_stats['year_month']),
        y0=lower_bound,
        x1=max(monthly_stats['year_month']),
        y1=lower_bound,
        line=dict(color="red", width=1, dash="dash"),
        row=row, col=col
    )

fig.update_layout(height=800, width=1200, title_text="Time Series Analysis with Anomaly Detection",
                 showlegend=False)
fig.show()

# ----------------------------------------------------------------------------------
# 3. MULTIVARIATE ANALYSIS
# ----------------------------------------------------------------------------------

print("\n\n========== 3. MULTIVARIATE ANALYSIS ==========")

# Select numerical features for multivariate analysis
num_features = ['agent_age', 'agent_tenure_months', 'unique_proposal', 'unique_quotations', 
               'unique_customers', 'new_policy_count', 'ANBP_value', 'net_income',
               'number_of_policy_holders', 'number_of_cash_payment_policies']

# Pairplot of key features colored by target
print("Generating pairplot of key features...")
sns.pairplot(train_df[num_features[:6] + ['target']], hue='target', height=2.5, 
             plot_kws={'alpha': 0.5, 's': 20})
plt.suptitle('Pairplot of Key Features', y=1.02)
plt.savefig('pairplot.png')
plt.show()

# Principal Component Analysis (PCA) for dimensionality reduction
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Prepare data for PCA
X = train_df[num_features].copy()
X = X.fillna(X.mean())  # Handle any remaining missing values
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Apply PCA
pca = PCA(n_components=3)
pca_result = pca.fit_transform(X_scaled)

# Create a DataFrame for visualization
pca_df = pd.DataFrame({
    'PC1': pca_result[:, 0],
    'PC2': pca_result[:, 1],
    'PC3': pca_result[:, 2],
    'target': train_df['target']
})

# Plot PCA results
plt.figure(figsize=(12, 10))

# 3D PCA Plot
ax = plt.subplot(111, projection='3d')
scatter = ax.scatter(
    pca_df['PC1'], 
    pca_df['PC2'], 
    pca_df['PC3'],
    c=pca_df['target'],
    cmap='viridis',
    alpha=0.5
)

ax.set_xlabel('Principal Component 1')
ax.set_ylabel('Principal Component 2')
ax.set_zlabel('Principal Component 3')
ax.set_title('3D PCA of Numerical Features')

# Add a color bar
plt.colorbar(scatter, ax=ax, label='Target (0=NILL, 1=Active)')

plt.tight_layout()
plt.savefig('pca_3d.png')
plt.show()

# Display explained variance
print("\nPCA Explained Variance Ratio:")
print(pca.explained_variance_ratio_)
print(f"Total Explained Variance (3 components): {sum(pca.explained_variance_ratio_):.4f}")

# Feature importance in each principal component
pca_components = pd.DataFrame(
    pca.components_.T,
    columns=['PC1', 'PC2', 'PC3'],
    index=num_features
)

plt.figure(figsize=(12, 10))
sns.heatmap(pca_components, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('PCA Components Feature Importance')
plt.tight_layout()
plt.savefig('pca_components.png')
plt.show()

# Interactive 3D scatter plot with Plotly
fig = px.scatter_3d(
    pca_df,
    x='PC1',
    y='PC2',
    z='PC3',
    color='target',
    opacity=0.7,
    title='3D PCA - NILL vs Active Agents'
)
fig.update_layout(height=800, width=1000)
fig.show()

# ----------------------------------------------------------------------------------
# 4. AGENT TRAJECTORIES OVER TIME
# ----------------------------------------------------------------------------------

print("\n\n========== 4. AGENT TRAJECTORIES OVER TIME ==========")

# Select a random sample of agents for trajectory analysis
sample_agents = train_df['agent_code'].unique()[:20]

# Filter data for the sampled agents
agent_trajectories = train_df[train_df['agent_code'].isin(sample_agents)]

# Plot agent performance over time
plt.figure(figsize=(16, 10))

# New policy count trajectories
plt.subplot(2, 2, 1)
for agent in sample_agents:
    agent_data = agent_trajectories[agent_trajectories['agent_code'] == agent]
    if not agent_data.empty:
        plt.plot(agent_data['year_month'], agent_data['new_policy_count'], marker='o', label=f'Agent {agent}')
plt.title('New Policy Count Trajectories')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

# ANBP value trajectories
plt.subplot(2, 2, 2)
for agent in sample_agents:
    agent_data = agent_trajectories[agent_trajectories['agent_code'] == agent]
    if not agent_data.empty:
        plt.plot(agent_data['year_month'], agent_data['ANBP_value'], marker='o', label=f'Agent {agent}')
plt.title('ANBP Value Trajectories')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

# Net income trajectories
plt.subplot(2, 2, 3)
for agent in sample_agents:
    agent_data = agent_trajectories[agent_trajectories['agent_code'] == agent]
    if not agent_data.empty:
        plt.plot(agent_data['year_month'], agent_data['net_income'], marker='o', label=f'Agent {agent}')
plt.title('Net Income Trajectories')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

# Target trajectories (NILL vs Active)
plt.subplot(2, 2, 4)
for agent in sample_agents:
    agent_data = agent_trajectories[agent_trajectories['agent_code'] == agent]
    if not agent_data.empty:
        plt.plot(agent_data['year_month'], agent_data['target'], marker='o', label=f'Agent {agent}')
plt.title('Target Status Trajectories (0=NILL, 1=Active)')
plt.yticks([0, 1])
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('agent_trajectories.png')
plt.show()

# Analyze the first few months of agent performance
early_performance = train_df.copy()

# Get the first few months (up to 3) for each agent
early_performance['months_active'] = early_performance.groupby('agent_code')['year_month'].rank()
early_months = early_performance[early_performance['months_active'] <= 3]

# Calculate average performance in early months
early_avg_performance = early_months.groupby('agent_code').agg({
    'new_policy_count': 'mean',
    'ANBP_value': 'mean',
    'net_income': 'mean',
    'target': 'mean'
}).reset_index()

early_avg_performance['early_nill_rate'] = 1 - early_avg_performance['target']

# Get the latest performance status for each agent
latest_performance = train_df.sort_values(['agent_code', 'year_month']).groupby('agent_code').last().reset_index()

# Merge early and latest performance
performance_trajectory = pd.merge(early_avg_performance, latest_performance[['agent_code', 'target']], 
                                 on='agent_code', suffixes=('_early', '_latest'))

# Calculate transition probabilities
print("\nTransition Probabilities from Early Performance to Latest Status:")
transition_counts = performance_trajectory.groupby(['target_early', 'target_latest']).size().unstack(fill_value=0)
transition_probs = transition_counts.div(transition_counts.sum(axis=1), axis=0)
print(transition_probs)

# Plot relationship between early performance and latest status
plt.figure(figsize=(16, 10))

metrics = ['new_policy_count', 'ANBP_value', 'net_income', 'early_nill_rate']
titles = ['Avg New Policy Count in First 3 Months', 'Avg ANBP Value in First 3 Months',
         'Avg Net Income in First 3 Months', 'NILL Rate in First 3 Months']

for i, (metric, title) in enumerate(zip(metrics, titles)):
    plt.subplot(2, 2, i+1)
    sns.boxplot(x='target_latest', y=metric, data=performance_trajectory)
    plt.title(f'{title} vs Latest Status')
    plt.xlabel('Latest Status (0=NILL, 1=Active)')
    plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('early_vs_latest_performance.png')
plt.show()

# ----------------------------------------------------------------------------------
# 5. INNOVATIVE EDA - HIDDEN INSIGHTS
# ----------------------------------------------------------------------------------

print("\n\n========== 5. INNOVATIVE EDA - HIDDEN INSIGHTS ==========")

# 1. Analyze the impact of the gap between joining and first sale
train_df['days_to_first_sale'] = (train_df['first_policy_sold_month'] - train_df['agent_join_month']).dt.days

plt.figure(figsize=(12, 6))
sns.boxplot(x='target', y='days_to_first_sale', data=train_df)
plt.title('Days to First Sale vs Target')
plt.xlabel('Target (0=NILL, 1=Active)')
plt.ylabel('Days to First Sale')
plt.grid(True, alpha=0.3)
plt.savefig('days_to_first_sale.png')
plt.show()

# 2. Analyze activity patterns (proposals, quotations, customers) as predictors of NILL
plt.figure(figsize=(16, 12))

activity_metrics = [
    'unique_proposals_last_7_days', 'unique_proposals_last_15_days', 'unique_proposals_last_21_days',
    'unique_quotations_last_7_days', 'unique_quotations_last_15_days', 'unique_quotations_last_21_days',
    'unique_customers_last_7_days', 'unique_customers_last_15_days', 'unique_customers_last_21_days'
]

for i, metric in enumerate(activity_metrics):
    plt.subplot(3, 3, i+1)
    sns.boxplot(x='target', y=metric, data=train_df)
    plt.title(f'{metric} vs Target')
    plt.xlabel('Target (0=NILL, 1=Active)')
    plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('activity_metrics.png')
plt.show()

# 3. Calculate the decay in activity over time windows
train_df['proposal_decay_7_to_15'] = train_df['unique_proposals_last_7_days'] - train_df['unique_proposals_last_15_days']
train_df['proposal_decay_15_to_21'] = train_df['unique_proposals_last_15_days'] - train_df['unique_proposals_last_21_days']
train_df['quotation_decay_7_to_15'] = train_df['unique_quotations_last_7_days'] - train_df['unique_quotations_last_15_days']
train_df['quotation_decay_15_to_21'] = train_df['unique_quotations_last_15_days'] - train_df['unique_quotations_last_21_days']
train_df['customer_decay_7_to_15'] = train_df['unique_customers_last_7_days'] - train_df['unique_customers_last_15_days']
train_df['customer_decay_15_to_21'] = train_df['unique_customers_last_15_days'] - train_df['unique_customers_last_21_days']

decay_metrics = [
    'proposal_decay_7_to_15', 'proposal_decay_15_to_21',
    'quotation_decay_7_to_15', 'quotation_decay_15_to_21',
    'customer_decay_7_to_15', 'customer_decay_15_to_21'
]

plt.figure(figsize=(16, 10))

for i, metric in enumerate(decay_metrics):
    plt.subplot(2, 3, i+1)
    sns.boxplot(x='target', y=metric, data=train_df)
    plt.title(f'{metric} vs Target')
    plt.xlabel('Target (0=NILL, 1=Active)')
    plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('activity_decay.png')
plt.show()

# 4. Agent age group analysis
train_df['age_group'] = pd.cut(train_df['agent_age'], bins=[20, 30, 40, 50, 60, 100], 
                              labels=['20-30', '30-40', '40-50', '50-60', '60+'])

age_group_stats = train_df.groupby('age_group').agg({
    'new_policy_count': 'mean',
    'ANBP_value': 'mean',
    'net_income': 'mean',
    'target': 'mean'
}).reset_index()

age_group_stats['nill_rate'] = 1 - age_group_stats['target']

plt.figure(figsize=(16, 10))

metrics = ['new_policy_count', 'ANBP_value', 'net_income', 'nill_rate']
titles = ['Avg New Policy Count', 'Avg ANBP Value', 'Avg Net Income', 'NILL Rate']

for i, (metric, title) in enumerate(zip(metrics, titles)):
    plt.subplot(2, 2, i+1)
    plt.bar(age_group_stats['age_group'], age_group_stats[metric])
    plt.title(f'{title} by Age Group')
    plt.xticks(rotation=45)
    plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('age_group_analysis.png')
plt.show()

# 5. Performance decay analysis - how performance changes over an agent's tenure
performance_by_tenure = train_df.groupby('agent_tenure_months').agg({
    'new_policy_count': 'mean',
    'ANBP_value': 'mean',
    'net_income': 'mean',
    'target': 'mean'
}).reset_index()

performance_by_tenure['nill_rate'] = 1 - performance_by_tenure['target']

plt.figure(figsize=(16, 10))

for i, (metric, title) in enumerate(zip(metrics, titles)):
    plt.subplot(2, 2, i+1)
    plt.plot(performance_by_tenure['agent_tenure_months'], performance_by_tenure[metric], marker='o')
    plt.title(f'{title} by Tenure Months')
    plt.xlabel('Tenure (Months)')
    plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('performance_by_tenure.png')
plt.show()

# 6. Cash payment preferences analysis
plt.figure(figsize=(12, 6))
sns.scatterplot(
    data=train_df,
    x='number_of_policy_holders',
    y='number_of_cash_payment_policies',
    hue='target',
    alpha=0.5
)
plt.title('Cash Payment Policies vs Total Policy Holders')
plt.xlabel('Number of Policy Holders')
plt.ylabel('Number of Cash Payment Policies')
plt.grid(True, alpha=0.3)
plt.savefig('cash_payment_analysis.png')
plt.show()

# 7. Feature importance for target prediction using a simple model
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler

# Prepare features
features = [
    'agent_age', 'agent_tenure_months', 'days_to_first_sale', 'months_since_first_sale',
    'unique_proposals_last_7_days', 'unique_proposals_last_15_days', 'unique_proposals_last_21_days',
    'unique_proposal', 'unique_quotations_last_7_days', 'unique_quotations_last_15_days',
    'unique_quotations_last_21_days', 'unique_quotations', 'unique_customers_last_7_days',
    'unique_customers_last_15_days', 'unique_customers_last_21_days', 'unique_customers',
    'proposal_decay_7_to_15', 'proposal_decay_15_to_21', 'quotation_decay_7_to_15', 
    'quotation_decay_15_to_21', 'customer_decay_7_to_15', 'customer_decay_15_to_21',
    'proposal_to_quotation_ratio', 'quotation_to_policy_ratio', 'proposal_to_policy_ratio',
    'cash_payment_ratio']

# Fill missing values with medians
X = train_df[features].copy()
X = X.fillna(X.median())

# Create target variable
y = train_df['target'].copy()

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train a simple model
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train_scaled, y_train)

# Evaluate
y_pred_proba = rf.predict_proba(X_test_scaled)[:, 1]
auc = roc_auc_score(y_test, y_pred_proba)
print(f"Model AUC-ROC: {auc:.4f}")

# Get feature importances
feature_importances = pd.DataFrame({
    'Feature': features,
    'Importance': rf.feature_importances_
})
feature_importances = feature_importances.sort_values('Importance', ascending=False)

plt.figure(figsize=(12, 8))
sns.barplot(x='Importance', y='Feature', data=feature_importances.head(15))
plt.title('Top 15 Important Features for Predicting NILL Agents')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('feature_importance.png')
plt.show()

# 8. Survival analysis - time to first NILL status
from lifelines import KaplanMeierFitter

# Create a new dataframe for survival analysis
# We need to identify when an agent first becomes NILL
agent_survival = train_df.sort_values(['agent_code', 'year_month']).copy()
agent_survival['is_nill'] = 1 - agent_survival['target']
agent_survival['first_nill'] = agent_survival.groupby('agent_code')['is_nill'].cumsum() > 0
agent_survival['time_to_first_nill'] = agent_survival.groupby('agent_code')['year_month'].rank()

# Get the first occurrence of NILL for each agent
first_nill_events = agent_survival[agent_survival['first_nill'] & (agent_survival['is_nill'] == 1)].groupby('agent_code').first().reset_index()
first_nill_events['event'] = 1  # NILL occurred

# Get agents who never became NILL
never_nill_agents = set(agent_survival['agent_code'].unique()) - set(first_nill_events['agent_code'].unique())
never_nill_data = agent_survival[agent_survival['agent_code'].isin(never_nill_agents)].groupby('agent_code').last().reset_index()
never_nill_data['event'] = 0  # NILL never occurred (censored)

# Combine data
survival_data = pd.concat([
    first_nill_events[['agent_code', 'time_to_first_nill', 'event', 'agent_age']],
    never_nill_data[['agent_code', 'time_to_first_nill', 'event', 'agent_age']]
])

# Fit Kaplan-Meier model
kmf = KaplanMeierFitter()
kmf.fit(survival_data['time_to_first_nill'], event_observed=survival_data['event'])

# Plot survival curve
plt.figure(figsize=(12, 6))
kmf.plot_survival_function()
plt.title('Survival Curve - Time to First NILL Status')
plt.xlabel('Months Active')
plt.ylabel('Probability of Still Being Active')
plt.grid(True, alpha=0.3)
plt.savefig('survival_curve.png')
plt.show()

# Split by age group
survival_data['age_group'] = pd.cut(survival_data['agent_age'], bins=[20, 30, 40, 50, 60, 100], 
                                   labels=['20-30', '30-40', '40-50', '50-60', '60+'])

plt.figure(figsize=(12, 6))
for age_group in survival_data['age_group'].unique():
    if pd.isna(age_group):
        continue
    kmf = KaplanMeierFitter()
    mask = survival_data['age_group'] == age_group
    if mask.sum() > 5:  # Only plot if we have enough data
        kmf.fit(survival_data[mask]['time_to_first_nill'], event_observed=survival_data[mask]['event'], label=age_group)
        kmf.plot_survival_function()
        
plt.title('Survival Curves by Age Group - Time to First NILL Status')
plt.xlabel('Months Active')
plt.ylabel('Probability of Still Being Active')
plt.grid(True, alpha=0.3)
plt.savefig('survival_curve_by_age.png')
plt.show()

# 9. Funnel analysis - Conversion through the sales process
funnel_stages = ['unique_proposal', 'unique_quotations', 'new_policy_count']
funnel_data = train_df[funnel_stages].mean().reset_index()
funnel_data.columns = ['Stage', 'Average']
funnel_data['Stage'] = ['Proposals', 'Quotations', 'Policies']

plt.figure(figsize=(10, 6))
plt.bar(funnel_data['Stage'], funnel_data['Average'])
plt.title('Sales Funnel - Average per Agent')
plt.grid(True, alpha=0.3)
plt.savefig('sales_funnel.png')
plt.show()

# Calculate conversion rates between funnel stages
print("\nFunnel Conversion Rates:")
print(f"Proposal to Quotation: {(train_df['unique_quotations'].mean() / train_df['unique_proposal'].mean()) * 100:.2f}%")
print(f"Quotation to Policy: {(train_df['new_policy_count'].mean() / train_df['unique_quotations'].mean()) * 100:.2f}%")
print(f"Overall (Proposal to Policy): {(train_df['new_policy_count'].mean() / train_df['unique_proposal'].mean()) * 100:.2f}%")

# Compare funnel conversion rates between NILL and Active agents
nill_funnel = train_df[train_df['target'] == 0][funnel_stages].mean()
active_funnel = train_df[train_df['target'] == 1][funnel_stages].mean()

conversion_comparison = pd.DataFrame({
    'NILL Agents': nill_funnel,
    'Active Agents': active_funnel
}).reset_index()
conversion_comparison.columns = ['Stage', 'NILL Agents', 'Active Agents']
conversion_comparison['Stage'] = ['Proposals', 'Quotations', 'Policies']

plt.figure(figsize=(10, 6))
x = np.arange(len(conversion_comparison['Stage']))
width = 0.35

plt.bar(x - width/2, conversion_comparison['NILL Agents'], width, label='NILL Agents')
plt.bar(x + width/2, conversion_comparison['Active Agents'], width, label='Active Agents')

plt.title('Sales Funnel Comparison - NILL vs Active Agents')
plt.xticks(x, conversion_comparison['Stage'])
plt.legend()
plt.grid(True, alpha=0.3)
plt.savefig('funnel_comparison.png')
plt.show()

# 10. Clustering agents based on their behavior
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Select features for clustering
cluster_features = [
    'agent_age', 'agent_tenure_months',
    'unique_proposal', 'unique_quotations', 'unique_customers',
    'proposal_to_quotation_ratio', 'quotation_to_policy_ratio'
]

# Prepare data
X_cluster = train_df[cluster_features].copy()
X_cluster = X_cluster.fillna(X_cluster.median())
X_cluster_scaled = StandardScaler().fit_transform(X_cluster)

# Determine optimal number of clusters using the elbow method
inertia = []
k_range = range(2, 11)
for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X_cluster_scaled)
    inertia.append(kmeans.inertia_)

plt.figure(figsize=(10, 6))
plt.plot(k_range, inertia, marker='o')
plt.title('Elbow Method for Optimal k')
plt.xlabel('Number of Clusters')
plt.ylabel('Inertia')
plt.grid(True, alpha=0.3)
plt.savefig('elbow_method.png')
plt.show()

# Choose the number of clusters based on the elbow method
n_clusters = 4  # Adjust based on the elbow plot
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
train_df['cluster'] = kmeans.fit_predict(X_cluster_scaled)

# Analyze cluster characteristics
cluster_stats = train_df.groupby('cluster').agg({
    'agent_age': 'mean',
    'agent_tenure_months': 'mean',
    'unique_proposal': 'mean',
    'unique_quotations': 'mean',
    'unique_customers': 'mean',
    'new_policy_count': 'mean',
    'ANBP_value': 'mean',
    'net_income': 'mean',
    'target': 'mean'
}).reset_index()

cluster_stats['nill_rate'] = 1 - cluster_stats['target']

# Display cluster statistics
print("\nCluster Characteristics:")
display(cluster_stats)

# Plot key metrics by cluster
plt.figure(figsize=(16, 10))

metrics = ['new_policy_count', 'ANBP_value', 'net_income', 'nill_rate']
titles = ['Avg New Policy Count', 'Avg ANBP Value', 'Avg Net Income', 'NILL Rate']

for i, (metric, title) in enumerate(zip(metrics, titles)):
    plt.subplot(2, 2, i+1)
    plt.bar(cluster_stats['cluster'], cluster_stats[metric])
    plt.title(f'{title} by Cluster')
    plt.xlabel('Cluster')
    plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('cluster_metrics.png')
plt.show()

# Visualization of clusters in 2D space using PCA
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_cluster_scaled)

plt.figure(figsize=(12, 8))
scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=train_df['cluster'], cmap='viridis', alpha=0.5)
plt.title('Agent Clusters Visualized in 2D Space')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.colorbar(scatter, label='Cluster')
plt.grid(True, alpha=0.3)
plt.savefig('cluster_visualization.png')
plt.show()

# Show NILL rate by cluster
plt.figure(figsize=(10, 6))
nill_by_cluster = train_df.groupby('cluster')['target'].mean().reset_index()
nill_by_cluster['nill_rate'] = 1 - nill_by_cluster['target']
plt.bar(nill_by_cluster['cluster'], nill_by_cluster['nill_rate'])
plt.title('NILL Rate by Cluster')
plt.xlabel('Cluster')
plt.ylabel('NILL Rate')
plt.grid(True, alpha=0.3)
plt.savefig('nill_rate_by_cluster.png')
plt.show()

# Print summary of insights
print("\n\n========== SUMMARY OF KEY INSIGHTS ==========")
print("\n1. The overall NILL rate in the dataset is {:.2f}%.".format((1 - train_df['target'].mean()) * 100))
print("\n2. Key features that distinguish NILL agents from active agents include:")
for feature, importance in zip(feature_importances['Feature'].head(5), feature_importances['Importance'].head(5)):
    print(f"   - {feature}: {importance:.4f}")
print("\n3. Agents typically have a higher risk of becoming NILL in their early months.")
print("\n4. Time series analysis shows seasonal patterns in agent performance.")
print("\n5. Agent activity metrics (proposals, quotations, customers) in the last 7, 15, and 21 days are strong predictors of NILL status.")
print("\n6. The proposal-to-policy conversion ratio is a key indicator of agent success.")
print("\n7. Clustering identified distinct agent profiles with varying NILL rates.")
print("\n8. Agent age and tenure show significant correlations with performance and NILL risk.")

    'unique_customers_last_15_days', 'unique_customers_last_21_days', 'unique_customers',
    'proposal_decay_7_to_15', 'proposal_decay_15_to_21', 'quotation_decay_7_to_15', 
    'quotation_decay_15_to_21', 'customer_decay_7_to_15', 'customer_decay_15_to_21',
    'proposal_to_quotation_ratio', 'quotation_to_policy_ratio', 'proposal_to_policy_ratio',
    'cash_payment_ratio'

IndentationError: unexpected indent (2315378397.py, line 939)